In [2]:
import requests
from bs4 import BeautifulSoup
page = requests.get("https://www.nhtsa.gov/node/97996/251")
page

<Response [200]>

In [3]:
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup.prettify())

In [4]:
# Find all hyperlink in the website
hpl= soup.find_all('a',href=True)

In [5]:
#Find all the link to each year
dl={} # variable that contains link to each year
for ele in hpl:
    e_text=ele.get_text()
    try:
        year=int(e_text)
        if (year>2013 and year <2019):
            print(year)
            print(ele['href'])
            dl[year]=ele['href']
    except: continue
    

2009
https://www.nhtsa.gov/node/97996/117196
2010
https://www.nhtsa.gov/node/97996/117201
2011
https://www.nhtsa.gov/node/97996/117206
2012
https://www.nhtsa.gov/node/97996/117211
2013
https://www.nhtsa.gov/node/97996/117216
2014
https://www.nhtsa.gov/node/97996/117221
2015
https://www.nhtsa.gov/node/97996/117226
2016
https://www.nhtsa.gov/node/97996/117231
2017
https://www.nhtsa.gov/node/97996/117236
2018
https://www.nhtsa.gov/node/97996/176766


## Download zip file of each year

In [7]:
import wget
for year in dl.keys():
    print(year)
    # find and select the national link
    page1 = requests.get(dl[year])
    soup1 = BeautifulSoup(page1.content, 'html.parser')
    hpl1= soup1.find_all('a',href=True)
    sublink=''
    for ele in hpl1:
        if ele.get_text()=='National':
            sublink=ele['href']
            break
    
    #find and select the csv file
    page1_1 = requests.get(sublink)
    soup1_1 = BeautifulSoup(page1_1.content, 'html.parser')
    hpl1_1= soup1_1.find_all('a',href=True)
    
    dll=''
    for ele in hpl1_1:
        if ele.get_text().endswith('SAS.zip'):
            dll=ele['href']
            break
    
    # Download file to the Data sets folder
    wget.download(dll,out='./Datasets')
    print('')

2009
100% [..........................................................................] 9707266 / 9707266
2010
100% [........................................................................] 13810722 / 13810722
2011
100% [........................................................................] 14099035 / 14099035
2012
100% [........................................................................] 15029317 / 15029317
2013
100% [........................................................................] 25191966 / 25191966
2014
100% [........................................................................] 25401135 / 25401135
2015
100% [........................................................................] 28731973 / 28731973
2016
100% [........................................................................] 21821873 / 21821873
2017
100% [........................................................................] 22270170 / 22270170
2018
100% [............................................

# Unzip files

In [8]:
from zipfile import ZipFile
import os

In [10]:
filelist=os.listdir('./Datasets')
dict_file=dict(zip(dl.keys(),filelist))
dict_file

{2009: 'FARS2009NationalSAS.zip',
 2010: 'FARS2010NationalSAS.zip',
 2011: 'FARS2011NationalSAS.zip',
 2012: 'FARS2012NationalSAS.zip',
 2013: 'FARS2013NationalSAS.zip',
 2014: 'FARS2014NationalSAS.zip',
 2015: 'FARS2015NationalSAS.zip',
 2016: 'FARS2016NationalSAS.zip',
 2017: 'FARS2017NationalSAS.zip',
 2018: 'FARS2018NationalSAS.zip'}

In [11]:
for year,fname in dict_file.items():
    path='./Datasets/'+fname
    zip_ref=ZipFile(path,'r')
    zip_ref.extractall('./Datasets/'+str(year))
    zip_ref.close()
    os. remove(path) # remove zip file

# Mapping and then convert to csv file

In [12]:
import pandas as pd
import pyreadstat as prs

In [31]:
os.getcwd()
files=os.listdir('.\Datasets\\2014')
print(files)

['accident.sas7bdat', 'acc_aux.sas7bdat', 'cevent.sas7bdat', 'damage.sas7bdat', 'distract.sas7bdat', 'drimpair.sas7bdat', 'factor.sas7bdat', 'Format14.sas', 'formats.sas7bcat', 'maneuver.sas7bdat', 'miacc.sas7bdat', 'midrvacc.sas7bdat', 'miper.sas7bdat', 'nmcrash.sas7bdat', 'nmimpair.sas7bdat', 'nmprior.sas7bdat', 'parkwork.sas7bdat', 'pbtype.sas7bdat', 'person.sas7bdat', 'per_aux.sas7bdat', 'safetyeq.sas7bdat', 'vehicle.sas7bdat', 'veh_aux.sas7bdat', 'vevent.sas7bdat', 'violatn.sas7bdat', 'vision.sas7bdat', 'vsoe.sas7bdat']


In [33]:
files.remove('Format14.sas')
files.remove('formats.sas7bcat')
print(files)

['accident.sas7bdat', 'acc_aux.sas7bdat', 'cevent.sas7bdat', 'damage.sas7bdat', 'distract.sas7bdat', 'drimpair.sas7bdat', 'factor.sas7bdat', 'maneuver.sas7bdat', 'miacc.sas7bdat', 'midrvacc.sas7bdat', 'miper.sas7bdat', 'nmcrash.sas7bdat', 'nmimpair.sas7bdat', 'nmprior.sas7bdat', 'parkwork.sas7bdat', 'pbtype.sas7bdat', 'person.sas7bdat', 'per_aux.sas7bdat', 'safetyeq.sas7bdat', 'vehicle.sas7bdat', 'veh_aux.sas7bdat', 'vevent.sas7bdat', 'violatn.sas7bdat', 'vision.sas7bdat', 'vsoe.sas7bdat']


In [34]:
def union_data(fname):
    df_final=pd.DataFrame()
    for year in range(2014,2019):
        path='.\Datasets\\'+str(year)
        name=''
        files=os.listdir(path)
        for file in files:
            if file.find(fname)!=-1:
                name = file
                break
        name
        path_f = path + '\\'+name
        path_c = path + '\\formats.sas7bcat'
        print(path_f,path_c)
        df= prs.read_sas7bdat(path_f, catalog_file=path_c, formats_as_category=True)[0]
        #print(df.head())
        #pd.read_csv(path,encoding = "ISO-8859-1", low_memory=False)
        df['YEAR']=year
        print(df.shape)
        df_final=pd.concat([df_final,df])
        del df
    return df_final
union_data(files[0]).head()

.\Datasets\2014\accident.sas7bdat .\Datasets\2014\formats.sas7bcat
(30056, 50)
.\Datasets\2015\accident.sas7bdat .\Datasets\2015\formats.sas7bcat
(32538, 52)
.\Datasets\2016\accident.sas7bdat .\Datasets\2016\formats.sas7bcat
(34748, 52)
.\Datasets\2017\accident.sas7bdat .\Datasets\2017\formats.sas7bcat
(34560, 52)
.\Datasets\2018\accident.sas7bdat .\Datasets\2018\formats.sas7bcat
(33654, 52)


,STATE,ST_CASE,VE_TOTAL,VE_FORMS,PVH_INVL,PEDS,PERNOTMVIT,PERMVIT,PERSONS,COUNTY,...,HOSP_HR,HOSP_MN,CF1,CF2,CF3,FATALS,DRUNK_DR,RUR_URB,FUNC_SYS,RD_OWNER
0,Alabama,10001.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,71.0,...,Unknown,Unknown EMS Hospital Arrival Time,None,None,None,1.0,1.0,NaN,NaN,NaN
1,Alabama,10002.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,59.0,...,Unknown,Unknown EMS Hospital Arrival Time,None,None,None,1.0,1.0,NaN,NaN,NaN
2,Alabama,10003.0,2.0,2.0,0.0,0.0,0.0,7.0,7.0,125.0,...,Unknown,Unknown EMS Hospital Arrival Time,None,None,None,2.0,0.0,NaN,NaN,NaN
3,Alabama,10004.0,3.0,3.0,0.0,0.0,0.0,5.0,5.0,121.0,...,Unknown,Unknown EMS Hospital Arrival Time,None,None,None,1.0,0.0,NaN,NaN,NaN
4,Alabama,10005.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,43.0,...,Not Applicable (Not Transported),Not Applicable (Not Transported),None,None,None,1.0,0.0,NaN,NaN,NaN


In [59]:
for fname in files:
    df=union_data(fname)
    na_cols=df.isna().any()[df.isna().any()==True].index
    print(na_cols)
    df.drop(columns=na_cols,inplace=True)
    try: 
        df=df[df.STATE=='Texas'].copy() # choose only Texas
    except: continue
    print(df.shape)
    df.to_csv('.\Datasets\\'+fname.split('.')[0]+'_5ys.csv',index=False)

.\Datasets\2014\accident.sas7bdat .\Datasets\2014\formats.sas7bcat
(30056, 50)
.\Datasets\2015\accident.sas7bdat .\Datasets\2015\formats.sas7bcat
(32538, 52)
.\Datasets\2016\accident.sas7bdat .\Datasets\2016\formats.sas7bcat
(34748, 52)
.\Datasets\2017\accident.sas7bdat .\Datasets\2017\formats.sas7bcat
(34560, 52)
.\Datasets\2018\accident.sas7bdat .\Datasets\2018\formats.sas7bcat
(33654, 52)
Index(['ROAD_FNC', 'RUR_URB', 'FUNC_SYS', 'RD_OWNER'], dtype='object')
(16464, 49)
.\Datasets\2014\acc_aux.sas7bdat .\Datasets\2014\formats.sas7bcat
(30056, 41)
.\Datasets\2015\acc_aux.sas7bdat .\Datasets\2015\formats.sas7bcat
(32538, 41)
.\Datasets\2016\acc_aux.sas7bdat .\Datasets\2016\formats.sas7bcat
(34748, 42)
.\Datasets\2017\acc_aux.sas7bdat .\Datasets\2017\formats.sas7bcat
(34560, 42)
.\Datasets\2018\acc_aux.sas7bdat .\Datasets\2018\formats.sas7bcat
(33654, 42)
Index(['A_WEATHER'], dtype='object')
(16464, 41)
.\Datasets\2014\cevent.sas7bdat .\Datasets\2014\formats.sas7bcat
(91214, 9)
.\Datas

(7802, 11)
Index(['MSAFEQMT', 'NMHELMET', 'NMPROPAD', 'NMOTHPRO', 'NMREFCLO', 'NMLIGHT',
       'NMOTHPRE'],
      dtype='object')
(3544, 5)
.\Datasets\2014\vehicle.sas7bdat .\Datasets\2014\formats.sas7bcat
(44950, 103)
.\Datasets\2015\vehicle.sas7bdat .\Datasets\2015\formats.sas7bcat
(49478, 103)
.\Datasets\2016\vehicle.sas7bdat .\Datasets\2016\formats.sas7bcat
(52714, 106)
.\Datasets\2017\vehicle.sas7bdat .\Datasets\2017\formats.sas7bcat
(53128, 106)
.\Datasets\2018\vehicle.sas7bdat .\Datasets\2018\formats.sas7bcat
(51872, 108)
Index(['PREV_SUS', 'TRLR1VIN', 'TRLR2VIN', 'TRLR3VIN', 'PREV_SUS1',
       'PREV_SUS2', 'PREV_SUS3'],
      dtype='object')
(25716, 102)
.\Datasets\2014\veh_aux.sas7bdat .\Datasets\2014\formats.sas7bcat
(44950, 16)
.\Datasets\2015\veh_aux.sas7bdat .\Datasets\2015\formats.sas7bcat
(49478, 16)
.\Datasets\2016\veh_aux.sas7bdat .\Datasets\2016\formats.sas7bcat
(52714, 17)
.\Datasets\2017\veh_aux.sas7bdat .\Datasets\2017\formats.sas7bcat
(53128, 17)
.\Datasets\2018

# map county and city

In [60]:
# accident
df=pd.read_csv('.\Datasets\\accident_5ys.csv')
df.head()

C:\Users\DuongHung\anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (16,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,STATE,ST_CASE,VE_TOTAL,VE_FORMS,PVH_INVL,PEDS,PERNOTMVIT,PERMVIT,PERSONS,COUNTY,...,NOT_MIN,ARR_HOUR,ARR_MIN,HOSP_HR,HOSP_MN,CF1,CF2,CF3,FATALS,DRUNK_DR
0,Texas,480001.0,2.0,2.0,0.0,1.0,1.0,2.0,2.0,29.0,...,Unknown if Notified,Unknown EMS Scene Arrival Hour,Unknown if Arrived,Not Applicable (Not Transported),Not Applicable (Not Transported),Recent/Previous Crash scene Nearby,None,None,1.0,0.0
1,Texas,480002.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,113.0,...,Unknown,Unknown EMS Scene Arrival Hour,Unknown EMS Scene Arrival Minutes,Unknown,Unknown EMS Hospital Arrival Time,None,None,None,1.0,1.0
2,Texas,480003.0,2.0,2.0,0.0,0.0,0.0,4.0,4.0,113.0,...,11,3:00pm-3:59pm,16,3:00pm-3:59pm,33,None,None,None,2.0,0.0
3,Texas,480004.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,157.0,...,Unknown if Notified,Unknown EMS Scene Arrival Hour,Unknown if Arrived,Not Applicable (Not Transported),Not Applicable (Not Transported),None,None,None,1.0,1.0
4,Texas,480005.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,177.0,...,Unknown if Notified,Unknown EMS Scene Arrival Hour,Unknown if Arrived,Not Applicable (Not Transported),Not Applicable (Not Transported),None,None,None,1.0,0.0


In [86]:
df.columns

Index(['STATE', 'ST_CASE', 'VE_TOTAL', 'VE_FORMS', 'PVH_INVL', 'PEDS',
       'PERNOTMVIT', 'PERMVIT', 'PERSONS', 'COUNTY', 'CITY', 'DAY', 'MONTH',
       'YEAR', 'DAY_WEEK', 'HOUR', 'MINUTE', 'NHS', 'ROUTE', 'TWAY_ID',
       'TWAY_ID2', 'MILEPT', 'LATITUDE', 'LONGITUD', 'SP_JUR', 'HARM_EV',
       'MAN_COLL', 'RELJCT1', 'RELJCT2', 'TYP_INT', 'WRK_ZONE', 'REL_ROAD',
       'LGT_COND', 'WEATHER1', 'WEATHER2', 'WEATHER', 'SCH_BUS', 'RAIL',
       'NOT_HOUR', 'NOT_MIN', 'ARR_HOUR', 'ARR_MIN', 'HOSP_HR', 'HOSP_MN',
       'CF1', 'CF2', 'CF3', 'FATALS', 'DRUNK_DR'],
      dtype='object')

In [61]:
GSA=pd.read_excel('.\Datasets\\GLCUnitedstatesJune102020.xlsx')
GSA.head()

,Territory,State Name,State Code,City Code,City Name,County Code,County Name,Country Code,Old City Name,Date Record Added,Duty Station Code
0,U,ALABAMA,1,10,ABBEVILLE,67,HENRY,840,NaN,NaT,10010067
1,U,ALABAMA,1,50,ALBERTVILLE,95,MARSHALL,840,NaN,NaT,10050095
2,U,ALABAMA,1,60,ALEXANDER CITY,123,TALLAPOOSA,840,NaN,NaT,10060123
3,U,ALABAMA,1,70,ALICEVILLE,107,PICKENS,840,NaN,NaT,10070107
4,U,ALABAMA,1,90,ANDALUSIA,39,COVINGTON,840,NaN,NaT,10090039


In [62]:
GSA_texas=GSA[GSA['State Name']=='TEXAS']
GSA_texas.head()

,Territory,State Name,State Code,City Code,City Name,County Code,County Name,Country Code,Old City Name,Date Record Added,Duty Station Code
32966,U,TEXAS,48,10,ABBOTT,217,HILL,840,NaN,NaT,480010217
32967,U,TEXAS,48,20,ABERNATHY,189,HALE,840,NaN,NaT,480020189
32968,U,TEXAS,48,20,ABERNATHY,303,LUBBOCK,840,NaN,NaT,480020303
32969,U,TEXAS,48,30,ABILENE,253,JONES,840,NaN,NaT,480030253
32970,U,TEXAS,48,30,ABILENE,441,TAYLOR,840,NaN,NaT,480030441


In [81]:
county_code=GSA_texas[['County Code','County Name']].copy().set_index('County Code')
county_code=county_code.to_dict()['County Name']

In [83]:
df['COUNTY']=df.COUNTY.astype('Int32').map(county_code)

In [84]:
city_code=GSA_texas[['City Code','City Name']].copy().set_index('City Code')
city_code=city_code.to_dict()['City Name']
city_code

{10: 'ABBOTT',
 20: 'ABERNATHY',
 30: 'ABILENE',
 23: 'ACALA',
 25: 'ACE',
 29: 'ACKERLY',
 34: 'ACTON',
 28: 'ADAMSVILLE',
 32: 'ADDICKS',
 31: 'ADDISON',
 33: 'ADKINS',
 35: 'ADRIAN',
 36: 'AFTON',
 40: 'AGUA DULCE',
 45: 'AGUILARES',
 48: 'AIKEN',
 50: 'ALAMO',
 60: 'ALAMO HEIGHTS',
 65: 'ALANREED',
 70: 'ALBA',
 80: 'ALBANY',
 83: 'ALBERT',
 85: 'ALEDO',
 90: 'ALICE',
 89: 'ALICE SOUTHWEST',
 91: 'ALIEF',
 92: 'ALLEN',
 95: 'ALLISON',
 96: 'ALMA',
 100: 'ALPINE',
 104: 'ALTAIR',
 110: 'ALTO',
 119: 'ALTOGA',
 114: 'ALTON',
 120: 'ALVARADO',
 130: 'ALVIN',
 140: 'ALVORD',
 150: 'AMARILLO',
 155: 'AMBROSE',
 157: 'AMES',
 160: 'AMHERST',
 170: 'ANAHUAC',
 175: 'ANDERSON',
 176: 'ANDERSON MILL',
 180: 'ANDREWS',
 190: 'ANGLETON',
 196: 'ANGUS',
 200: 'ANNA',
 208: 'ANNETH NORTH',
 210: 'ANNONA',
 220: 'ANSON',
 223: 'ANTELOPE',
 225: 'ANTHONY',
 230: 'ANTON',
 231: 'APPLE SPRINGS',
 232: 'APPLEBY',
 237: 'AQUILLA',
 240: 'ARANSAS PASS',
 249: 'ARCADIA',
 250: 'FRISCO',
 253: 'ARCOLA',

In [85]:
df['CITY']=df.CITY.astype('Int32').map(city_code)
df.head()

,STATE,ST_CASE,VE_TOTAL,VE_FORMS,PVH_INVL,PEDS,PERNOTMVIT,PERMVIT,PERSONS,COUNTY,...,NOT_MIN,ARR_HOUR,ARR_MIN,HOSP_HR,HOSP_MN,CF1,CF2,CF3,FATALS,DRUNK_DR
0,Texas,480001.0,2.0,2.0,0.0,1.0,1.0,2.0,2.0,BEXAR,...,Unknown if Notified,Unknown EMS Scene Arrival Hour,Unknown if Arrived,Not Applicable (Not Transported),Not Applicable (Not Transported),Recent/Previous Crash scene Nearby,None,None,1.0,0.0
1,Texas,480002.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,DALLAS,...,Unknown,Unknown EMS Scene Arrival Hour,Unknown EMS Scene Arrival Minutes,Unknown,Unknown EMS Hospital Arrival Time,None,None,None,1.0,1.0
2,Texas,480003.0,2.0,2.0,0.0,0.0,0.0,4.0,4.0,DALLAS,...,11,3:00pm-3:59pm,16,3:00pm-3:59pm,33,None,None,None,2.0,0.0
3,Texas,480004.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,FORT BEND,...,Unknown if Notified,Unknown EMS Scene Arrival Hour,Unknown if Arrived,Not Applicable (Not Transported),Not Applicable (Not Transported),None,None,None,1.0,1.0
4,Texas,480005.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,GONZALES,...,Unknown if Notified,Unknown EMS Scene Arrival Hour,Unknown if Arrived,Not Applicable (Not Transported),Not Applicable (Not Transported),None,None,None,1.0,0.0


In [87]:
s_col=['STATE', 'ST_CASE', 'VE_TOTAL', 'VE_FORMS', 'PVH_INVL', 'PEDS',
       'PERNOTMVIT', 'PERMVIT', 'PERSONS', 'COUNTY', 'CITY', 'DAY', 'MONTH',
       'YEAR', 'DAY_WEEK', 'HOUR', 'MINUTE', 'NHS', 'ROUTE', 'LATITUDE', 'LONGITUD', 'SP_JUR', 'HARM_EV',
       'MAN_COLL', 'RELJCT1', 'RELJCT2', 'TYP_INT', 'WRK_ZONE', 'REL_ROAD',
       'LGT_COND', 'WEATHER1', 'WEATHER2', 'WEATHER', 'SCH_BUS', 'RAIL',
       'CF1', 'CF2', 'CF3', 'FATALS', 'DRUNK_DR']

In [88]:
df[s_col].to_csv('.\Datasets\\accident_5ys.csv',index=False)

In [89]:
# person
df=pd.read_csv('.\Datasets\\person_5ys.csv')
df.head()

C:\Users\DuongHung\anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,STATE,ST_CASE,VE_FORMS,VEH_NO,PER_NO,STR_VEH,COUNTY,DAY,MONTH,HOUR,...,LAG_HRS,LAG_MINS,P_SF1,P_SF2,P_SF3,WORK_INJ,HISPANIC,RACE,LOCATION,YEAR
0,Texas,480001.0,2.0,0.0,1.0,1.0,29.0,1,January,4:00am-4:59am,...,0.0,29.0,None,None,None,No,Mexican,White,"Not at Intersection - On Roadway, Not in Marke...",2014
1,Texas,480001.0,2.0,1.0,1.0,0.0,29.0,1,January,4:00am-4:59am,...,999.0,99.0,None,None,None,Not Applicable (not a fatality),Not A Fatality (not Applicable),Not a Fatality (not Applicable),Occupant of a Motor Vehicle,2014
2,Texas,480001.0,2.0,1.0,2.0,0.0,29.0,1,January,4:00am-4:59am,...,999.0,99.0,None,None,None,Not Applicable (not a fatality),Not A Fatality (not Applicable),Not a Fatality (not Applicable),Occupant of a Motor Vehicle,2014
3,Texas,480002.0,1.0,1.0,1.0,0.0,113.0,1,January,0:00am-0:59am,...,1.0,48.0,None,None,None,No,Non-Hispanic,Black,Occupant of a Motor Vehicle,2014
4,Texas,480003.0,2.0,1.0,1.0,0.0,113.0,1,January,3:00pm-3:59pm,...,999.0,99.0,None,None,None,Not Applicable (not a fatality),Not A Fatality (not Applicable),Not a Fatality (not Applicable),Occupant of a Motor Vehicle,2014


In [91]:
df['COUNTY']=df.COUNTY.astype('Int32').map(county_code)

In [92]:
df.columns

Index(['STATE', 'ST_CASE', 'VE_FORMS', 'VEH_NO', 'PER_NO', 'STR_VEH', 'COUNTY',
       'DAY', 'MONTH', 'HOUR', 'MINUTE', 'HARM_EV', 'MAN_COLL', 'SCH_BUS',
       'AGE', 'SEX', 'PER_TYP', 'INJ_SEV', 'SEAT_POS', 'REST_USE', 'REST_MIS',
       'AIR_BAG', 'EJECTION', 'EJ_PATH', 'EXTRICAT', 'DRINKING', 'ALC_DET',
       'ALC_STATUS', 'ATST_TYP', 'ALC_RES', 'DRUGS', 'DRUG_DET', 'DSTATUS',
       'HOSPITAL', 'DOA', 'DEATH_DA', 'DEATH_MO', 'DEATH_YR', 'DEATH_HR',
       'DEATH_MN', 'DEATH_TM', 'LAG_HRS', 'LAG_MINS', 'P_SF1', 'P_SF2',
       'P_SF3', 'WORK_INJ', 'HISPANIC', 'RACE', 'LOCATION', 'YEAR'],
      dtype='object')

In [93]:
s_col=['STATE', 'ST_CASE', 'VE_FORMS', 'VEH_NO', 'PER_NO', 'STR_VEH', 'COUNTY',
       'DAY', 'MONTH', 'HOUR', 'MINUTE', 'HARM_EV', 'MAN_COLL', 'SCH_BUS',
       'AGE', 'SEX', 'PER_TYP', 'INJ_SEV', 'SEAT_POS', 'REST_USE', 'REST_MIS',
       'AIR_BAG', 'EJECTION', 'EJ_PATH', 'EXTRICAT', 'DRINKING', 
       'ALC_RES', 'DRUGS', 'DSTATUS',
       'HOSPITAL', 'DOA', 'P_SF1', 'P_SF2',
       'P_SF3', 'WORK_INJ', 'HISPANIC', 'RACE', 'LOCATION', 'YEAR']

In [94]:
df[s_col].to_csv('.\Datasets\\person_5ys.csv',index=False)

In [95]:
# vehicle
df=pd.read_csv('.\Datasets\\vehicle_5ys.csv')
df.head()

C:\Users\DuongHung\anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,STATE,ST_CASE,VEH_NO,VE_FORMS,NUMOCCS,DAY,MONTH,HOUR,MINUTE,HARM_EV,...,VTCONT_F,P_CRASH1,P_CRASH2,P_CRASH3,PCRASH4,PCRASH5,ACC_TYPE,DEATHS,DR_DRINK,YEAR
0,Texas,480001.0,1.0,2.0,02,1,January,4:00am-4:59am,22,Motor Vehicle In-Transport,...,No Controls,Going Straight,Other vehicle stopped,Unknown,Tracking,Stayed in original travel lane,Specifics Other,0.0,No,2014
1,Texas,480001.0,2.0,2.0,None,1,January,4:00am-4:59am,22,Motor Vehicle In-Transport,...,No Controls,No Driver Present / Unknown if Driver Present,Traveling in same direction with higher speed,No Driver Present / Unknown if Driver Present,No Driver Present / Unknown if Driver Present,No Driver Present / Unknown if Driver Present,Specifics Other,0.0,No,2014
2,Texas,480002.0,1.0,1.0,01,1,January,0:00am-0:59am,20,Tree (Standing Only),...,No Controls,Turning Left,Off the edge of the road on the left side,Unknown,Tracking,Departed roadway,Specifics Other,1.0,Yes,2014
3,Texas,480003.0,1.0,2.0,02,1,January,3:00pm-3:59pm,11,Motor Vehicle In-Transport,...,Device Functioning Properly,Turning Left,Turning left at junction,Unknown,Tracking,Stayed in original travel lane,Initial Opposite Directions (Left/Right),0.0,No,2014
4,Texas,480003.0,2.0,2.0,02,1,January,3:00pm-3:59pm,11,Motor Vehicle In-Transport,...,No Controls,Going Straight,"From driveway, turning into opposite direction",Unknown,Tracking,Stayed in original travel lane,Initial Opposite Directions (Going Straight),2.0,No,2014


In [99]:
print(df.columns[:50])
print(df.columns[50:])

Index(['STATE', 'ST_CASE', 'VEH_NO', 'VE_FORMS', 'NUMOCCS', 'DAY', 'MONTH',
       'HOUR', 'MINUTE', 'HARM_EV', 'MAN_COLL', 'UNITTYPE', 'HIT_RUN',
       'REG_STAT', 'OWNER', 'MAKE', 'MODEL', 'MAK_MOD', 'BODY_TYP', 'MOD_YEAR',
       'VIN', 'VIN_1', 'VIN_2', 'VIN_3', 'VIN_4', 'VIN_5', 'VIN_6', 'VIN_7',
       'VIN_8', 'VIN_9', 'VIN_10', 'VIN_11', 'VIN_12', 'TOW_VEH', 'J_KNIFE',
       'MCARR_I1', 'MCARR_I2', 'MCARR_ID', 'GVWR', 'V_CONFIG', 'CARGO_BT',
       'HAZ_INV', 'HAZ_PLAC', 'HAZ_ID', 'HAZ_CNO', 'HAZ_REL', 'BUS_USE',
       'SPEC_USE', 'EMER_USE', 'TRAV_SP'],
      dtype='object')
Index(['UNDERIDE', 'ROLLOVER', 'ROLINLOC', 'IMPACT1', 'DEFORMED', 'TOWED',
       'M_HARM', 'VEH_SC1', 'VEH_SC2', 'FIRE_EXP', 'DR_PRES', 'L_STATE',
       'DR_ZIP', 'L_STATUS', 'L_TYPE', 'CDL_STAT', 'L_ENDORS', 'L_COMPL',
       'L_RESTRI', 'DR_HGT', 'DR_WGT', 'PREV_ACC', 'PREV_DWI', 'PREV_SPD',
       'PREV_OTH', 'FIRST_MO', 'FIRST_YR', 'LAST_MO', 'LAST_YR', 'SPEEDREL',
       'DR_SF1', 'DR_SF2', 'DR_S

In [100]:
s_col=['STATE', 'ST_CASE', 'VEH_NO', 'VE_FORMS', 'NUMOCCS', 'DAY', 'MONTH',
       'HOUR', 'MINUTE', 'HARM_EV', 'MAN_COLL', 'UNITTYPE', 'HIT_RUN',
       'REG_STAT', 'OWNER', 'MAKE', 'MODEL', 'MAK_MOD', 'BODY_TYP', 'MOD_YEAR',
       'TOW_VEH', 'J_KNIFE','GVWR', 'V_CONFIG', 'CARGO_BT',
       'HAZ_INV', 'HAZ_PLAC', 'HAZ_ID', 'HAZ_CNO', 'HAZ_REL', 'BUS_USE',
       'SPEC_USE', 'EMER_USE', 'TRAV_SP','UNDERIDE', 'ROLLOVER', 'ROLINLOC', 'IMPACT1', 'DEFORMED', 'TOWED',
       'M_HARM', 'VEH_SC1', 'VEH_SC2', 'FIRE_EXP', 'DR_PRES', 'L_STATE',
       'L_STATUS', 'L_TYPE', 'CDL_STAT', 'L_ENDORS', 'L_COMPL',
       'L_RESTRI', 'DR_HGT', 'DR_WGT', 'PREV_ACC', 'PREV_DWI', 'PREV_SPD',
       'PREV_OTH', 'FIRST_MO', 'FIRST_YR', 'LAST_MO', 'LAST_YR', 'SPEEDREL',
       'DR_SF1', 'DR_SF2', 'DR_SF3', 'DR_SF4', 'VTRAFWAY', 'VNUM_LAN',
       'VSPD_LIM', 'VALIGN', 'VPROFILE', 'VPAVETYP', 'VSURCOND', 'VTRAFCON',
       'VTCONT_F', 'P_CRASH1', 'P_CRASH2', 'P_CRASH3', 'PCRASH4', 'PCRASH5',
       'ACC_TYPE', 'DEATHS', 'DR_DRINK', 'YEAR']

In [101]:
df[s_col].to_csv('.\Datasets\\vehicle_5ys.csv',index=False)